# 📈 KOSPI 시총 상위 10종목 데이터 자동화 노트북

Recent 1년간 **KOSPI 상위 10개 종목**의 주가 데이터를 수집하고:

- 거래량 0 제거 & 결측값 보간
- 일별 수익률 계산
- 거래량 상위 5일 추출

을 수행한 뒤 **Google 스프레드시트**에 업로드합니다.


## 🔐 Google 스프레드시트 API 인증 설정

1. **Google Cloud Console**에서 프로젝트 생성  
2. **APIs & Services → Enabled APIs & services**에서  
   - Google Sheets API  
   - Google Drive API  
   를 각각 **Enable**  
3. **APIs & Services → Credentials → Create credentials → Service account** 생성  
   - 역할: Editor  
4. 서비스 계정 생성 후 **Keys → Add key → JSON**  
   - `credentials.json` 다운로드  
5. 스프레드시트에서 **공유** → 서비스 계정 이메일(예: `xxx@yyy.iam.gserviceaccount.com`)에 편집 권한 부여  
6. 이 노트북과 동일 폴더에 `credentials.json` 배치  


In [1]:
# 필요한 패키지 설치 및 임포트
# !pip install yfinance pandas numpy gspread gspread_dataframe

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import gspread
from gspread_dataframe import set_with_dataframe



   ---- -----------------------------------  1/10 [oauthlib]
   ---- -----------------------------------  1/10 [oauthlib]
   ------------ ---------------------------  3/10 [rsa]
   -------------------- -------------------  5/10 [pyasn1-modules]
   -------------------- -------------------  5/10 [pyasn1-modules]
   ------------------------ ---------------  6/10 [google-auth]
   -------------------------------- -------  8/10 [gspread]
   ---------------------------------------- 10/10 [gspread_dataframe]



In [3]:
import yfinance as yf
from datetime import datetime, timedelta

# 1. 티커 딕셔너리
tickers = {
    'SamsungElec': '005930.KS',
    'SKhynix': '000660.KS',
    'HyundaiMotor': '005380.KS',
    'LGChem': '003550.KS',
    'SamsungSDI': '006400.KS',
    'POSCO': '005490.KS',
    'Naver': '035420.KS',
    'Kia': '000270.KS',
    'HyundaiMobis': '012330.KS',
    'LGEnergySol': '373220.KS'
}

# 2. 기간 설정
end   = datetime.today()
start = end - timedelta(days=365)

# 3. 데이터 다운로드
raw = yf.download(
    tickers=list(tickers.values()),
    start=start, end=end,
    group_by='ticker',
    auto_adjust=False
)

# 4. 각 티커별로 DataFrame 생성 후 리스트에 저장
frames = []
for name, symbol in tickers.items():
    df = raw[symbol].copy()   # Date 인덱스, 시가/고가/저가/종가/...
    df['Ticker'] = name       # 티커 이름을 컬럼으로 추가
    frames.append(df)

# ————————————————————————————————————————————————
# 5. concat → reset_index → set_index (MultiIndex)
price_df = pd.concat(frames)                # Date-index + 'Ticker' 컬럼
price_df = price_df.reset_index()           # Date 인덱스를 컬럼으로 이동
price_df = price_df.set_index(['Ticker','Date'])
price_df.index.names = ['Ticker','Date']    # 인덱스 레벨 이름 설정

# 6. 결과 확인
print(price_df.head())


[*********************100%***********************]  10 of 10 completed

Price                      Open     High      Low    Close     Adj Close  \
Ticker      Date                                                           
SamsungElec 2024-04-29  77400.0  77600.0  76200.0  76700.0  75419.117188   
            2024-04-30  77000.0  78500.0  76600.0  77500.0  76205.757812   
            2024-05-02  77600.0  78600.0  77300.0  78000.0  76697.406250   
            2024-05-03  79000.0  79000.0  77500.0  77600.0  76304.093750   
            2024-05-07  79600.0  81300.0  79400.0  81300.0  79942.304688   

Price                     Volume  
Ticker      Date                  
SamsungElec 2024-04-29  14664474  
            2024-04-30  19007007  
            2024-05-02  18900640  
            2024-05-03  13151889  
            2024-05-07  26238868  


In [8]:
# 1) 거래량 0 제거 & 결측치는 티커별로 forward-fill
clean = (
    price_df
    .loc[price_df['Volume'] != 0]    # Volume이 0인 로우 제외
    .groupby(level='Ticker')         # 'Ticker' 레벨별 그룹화
    .ffill()                         # 그룹별로 ffill
)

# 2) 티커별 일일 수익률(Return) 계산
clean['Return'] = (
    clean
    .groupby(level='Ticker')['Close']  # 티커별 Close
    .pct_change()                      # pct_change 적용
)

# 3) 거래량 상위 5일 추출
top5 = (
    clean
    .reset_index()                     # MultiIndex → 컬럼으로
    .sort_values(['Ticker','Volume'], ascending=[True,False])
    .groupby('Ticker', as_index=False)
    .head(5)
)

print("거래량 상위 5일:")
print(top5[['Ticker','Date','Volume']])


거래량 상위 5일:
Price        Ticker       Date    Volume
1974   HyundaiMobis 2024-06-14   1763305
2062   HyundaiMobis 2024-10-25    823915
2055   HyundaiMobis 2024-10-16    624192
2124   HyundaiMobis 2025-01-24    569055
1981   HyundaiMobis 2024-06-25    532626
500    HyundaiMotor 2024-05-22   3336634
546    HyundaiMotor 2024-07-26   2662741
703    HyundaiMotor 2025-03-25   2097056
656    HyundaiMotor 2025-01-10   1951806
517    HyundaiMotor 2024-06-17   1945849
1785            Kia 2024-08-30   3386834
1732            Kia 2024-06-17   2654199
1733            Kia 2024-06-18   2632809
1767            Kia 2024-08-05   2493835
1916            Kia 2025-03-21   2448199
930          LGChem 2025-02-28   1226185
815          LGChem 2024-09-03    563126
750          LGChem 2024-05-31    503669
828          LGChem 2024-09-25    488451
792          LGChem 2024-07-31    464155
2320    LGEnergySol 2024-11-15   1016392
2316    LGEnergySol 2024-11-11    969362
2293    LGEnergySol 2024-10-08    803706
2272 

In [7]:
# Google Sheets 업로드
SHEET_NAME = 'KOSPI_TOP10_1Y'
CREDS_FILE = 'credentials.json'

gc = gspread.service_account(filename=CREDS_FILE)
try:
    sh = gc.open(SHEET_NAME)
except gspread.SpreadsheetNotFound:
    sh = gc.create(SHEET_NAME)
    # 필요 시 실제 사용자 이메일로 공유: sh.share('you@example.com', perm_type='user', role='writer')

# 전체 데이터 시트
ws_all = sh.sheet1
set_with_dataframe(ws_all, clean.reset_index())

# 거래량 TOP5 시트
try:
    ws_top = sh.worksheet('TOP5_VOLUME')
except gspread.WorksheetNotFound:
    ws_top = sh.add_worksheet(title='TOP5_VOLUME', rows='100', cols='20')
set_with_dataframe(ws_top, top5)

print("Google 스프레드 시트 업로드 완료!")


FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'

## 🖥️ GUI 실행 파일 제작

자동화 스크립트를 **GUI**로 실행하려면:

```bash
pip install pyinstaller
```

`app.py` 파일 생성 후:

```bash
pyinstaller --onefile --noconsole app.py
```

- `--onefile`: 단일 실행 파일 생성  
- `--noconsole`: 콘솔 창 숨김  

`dist/app.exe` 실행 시 버튼 클릭만으로 작업이 수행됩니다.
